# Experimentos 18/10/23 - BayesFL

- Probar con todas las combinaciones de BN_FusionUnion y BN_FusionIntersection. 
- Experimentos en portátil MSI.
- Base más grande: `andes`.
- 4 clientes.
- 5000 instancias para cada cliente.
- Sin interleaving.

In [1]:
import warnings
warnings.filterwarnings('ignore')

from IPython.display import display

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import pingouin as pg

import glob
import os
import shutil

pd.options.display.max_info_rows = 30000000
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 100)

sns.set_theme(style='darkgrid')

# Tamaño figuras
plt.rcParams["figure.figsize"] = (20,8)
plt.rcParams["figure.dpi"] = 600

pd.options.mode.chained_assignment = None

# Paleta de colores
color = ['#2a9d8f', '#e9c46a', '#f4a261', '#e76f51', '#b13c47']

color2 = ['#264653', '#2a9d8f', '#e9c46a', '#f4a261', '#e76f51', '#b13c47']

In [2]:
# Join all of csv files of the folder ../resultados, mantaining the header of the first file
path = r'../results/Client/' # use your path
all_files = glob.glob(os.path.join(path, "*.csv"))

df_from_each_file = (pd.read_csv(f, sep=',') for f in all_files)
df = pd.concat(df_from_each_file, ignore_index=True)

df

,bbdd,algorithm,id,iteration,instances,threads,bdeu,SMHD,time(s)
0,alarm_50001,GES_BN_FusionIntersection_GES_BN_FusionInterse...,0,1,5000,8,-54570.524663,10,0.154
1,alarm_50001,GES_BN_FusionIntersection_GES_BN_FusionInterse...,0,2,5000,8,-53557.711947,7,0.039
2,alarm_50001,GES_BN_FusionIntersection_GES_BN_FusionInterse...,0,3,5000,8,-53557.711947,7,0.031
3,alarm_50001,GES_BN_FusionIntersection_GES_BN_FusionInterse...,0,4,5000,8,-53557.711947,7,0.037
4,alarm_50001,GES_BN_FusionIntersection_GES_BN_FusionInterse...,0,5,5000,8,-53557.711947,7,0.036
...,...,...,...,...,...,...,...,...,...
8023,win95pts_50004,pGES_BN_FusionUnion_None_BN_FusionUnion_fusion,3,1,5000,8,-47436.404946,64,0.014
8024,win95pts_50004,pGES_BN_FusionUnion_None_BN_FusionUnion_fusion,3,2,5000,8,-47460.319854,65,0.034
8025,win95pts_50004,pGES_BN_FusionUnion_None_BN_FusionUnion_fusion,3,3,5000,8,-47460.319854,65,0.025
8026,win95pts_50004,pGES_BN_FusionUnion_None_BN_FusionUnion_fusion,3,4,5000,8,-47460.319854,65,0.035


In [3]:
# Replace the GES_BN_FusionIntersection_GES_build of column algorithm with four columns separated with the _
df[['algorithm','fusionClient','refinement','fusionServer','epoch']] = df.algorithm.str.split("_",expand=True,).drop(columns=[1,4])

df[['bbdd','sample']] = df.bbdd.str.split("_",expand=True,)

# Delete bbdd diabetes and link
df = df[df.bbdd != 'diabetes']
df = df[df.bbdd != 'link']

df

,bbdd,algorithm,id,iteration,instances,threads,bdeu,SMHD,time(s),fusionClient,refinement,fusionServer,epoch,sample
0,alarm,GES,0,1,5000,8,-54570.524663,10,0.154,FusionIntersection,GES,FusionIntersection,build,50001
1,alarm,GES,0,2,5000,8,-53557.711947,7,0.039,FusionIntersection,GES,FusionIntersection,build,50001
2,alarm,GES,0,3,5000,8,-53557.711947,7,0.031,FusionIntersection,GES,FusionIntersection,build,50001
3,alarm,GES,0,4,5000,8,-53557.711947,7,0.037,FusionIntersection,GES,FusionIntersection,build,50001
4,alarm,GES,0,5,5000,8,-53557.711947,7,0.036,FusionIntersection,GES,FusionIntersection,build,50001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8023,win95pts,pGES,3,1,5000,8,-47436.404946,64,0.014,FusionUnion,None,FusionUnion,fusion,50004
8024,win95pts,pGES,3,2,5000,8,-47460.319854,65,0.034,FusionUnion,None,FusionUnion,fusion,50004
8025,win95pts,pGES,3,3,5000,8,-47460.319854,65,0.025,FusionUnion,None,FusionUnion,fusion,50004
8026,win95pts,pGES,3,4,5000,8,-47460.319854,65,0.035,FusionUnion,None,FusionUnion,fusion,50004


### BDeu

In [4]:
# Drop the rows with epoch != build
df2 = df[df['epoch'] == 'build']

# Only iteration 5
df2 = df2[(df2['iteration'] == 1) | (df2['iteration'] == 5)]

df_unstack = df2.drop(["sample", "id", "threads", "instances", "epoch", "SMHD", "time(s)"], axis=1)
df_unstack.groupby(["bbdd", "algorithm", "fusionClient", "fusionServer", "refinement", "iteration"], observed=True).mean().unstack(0)

bdeu  \
bbdd                                                                         alarm   
algorithm fusionClient       fusionServer       refinement iteration                 
GES       FusionIntersection FusionIntersection GES        1         -53993.785619   
                                                           5         -53653.577956   
                                                None       1         -53993.785619   
                                                           5         -53596.153387   
                             FusionUnion        GES        1         -53993.785619   
                                                           5         -53648.982757   
                                                None       1         -53993.785619   
                                                           5         -53605.576683   
          FusionUnion        FusionIntersection GES        1         -53993.785619   
                                                           5         -53671.570510   
                                                None       1         -53993.785619   
                                                           5         -53653.577956   
                             FusionUnion        GES        1         -53993.785619   
                                                           5         -53648.982757   
                                                None       1         -53993.785619   
                                                           5         -54030.288104   
pGES      FusionIntersection FusionIntersection GES        1         -53672.317608   
                                                           5         -53653.577956   
                                                None       1         -53651.755373   
                                                           5         -53596.153387   
                             FusionUnion        GES        1         -53672.317608   
                                                           5         -53672.317608   
                                                None       1         -53672.317608   
                                                           5         -53608.349299   
          FusionUnion        FusionIntersection GES        1         -53672.317608   
                                                           5         -53653.577956   
                                                None       1         -53672.317608   
                                                           5         -53653.577956   
                             FusionUnion        GES        1         -53672.317608   
                                                           5         -53672.317608   
                                                None       1         -53672.317608   
                                                           5         -53675.912854   

                                                                                     \
bbdd                                                                          andes   
algorithm fusionClient       fusionServer       refinement iteration                  
GES       FusionIntersection FusionIntersection GES        1         -471382.420224   
                                                           5         -471524.811981   
                                                None       1         -471382.420224   
                                                           5         -471377.105068   
                             FusionUnion        GES        1         -471382.420224   
                                                           5         -471377.937926   
                                                None       1         -471382.420224   
                                                           5         -471377.937926   
          FusionUnion        FusionIntersection GES        1         -471382.420224   
                       

In [39]:
df_unstack = df2.drop(["sample", "id", "threads", "instances", "epoch", "SMHD", "time(s)", "fusionClient", "fusionServer", "refinement"], axis=1)
df_unstack.groupby(["bbdd", "algorithm", "iteration"], observed=True).mean().unstack(0)

bdeu                                              \
bbdd                        alarm          andes         barley         child   
algorithm iteration                                                             
GES       1         -53993.785619 -471382.420224 -265927.284293 -62160.327550   
          5         -53688.588764 -471409.213226 -264157.178848 -61958.728871   
pGES      1         -53669.747329 -471377.937926 -264117.900969 -62041.499597   
          5         -53648.223078 -471372.494713 -263889.756592 -61979.345719   

                                                                 \
bbdd                    hailfinder         hepar2     insurance   
algorithm iteration                                               
GES       1         -252158.044952 -164689.532991 -67574.976828   
          5         -252337.690528 -164695.823762 -67650.498646   
pGES      1         -252174.472470 -164689.532991 -67626.908305   
          5         -252184.178104 -164691.729687 -67662.672891   

                                                                
bbdd                        mildew         water      win95pts  
algorithm iteration                                             
GES       1         -235357.914167 -65913.330040 -46896.283478  
          5         -235357.914167 -65903.459590 -46879.985591  
pGES      1         -235616.652494 -66773.993382 -46800.386186  
          5         -235551.967913 -65908.551927 -46834.118806

In [40]:
df_unstack = df2.drop(["sample", "id", "threads", "instances", "epoch", "SMHD", "time(s)","bbdd"], axis=1)
df_unstack.groupby(["iteration", "algorithm", "fusionClient", "fusionServer", "refinement"], observed=True).mean()

bdeu
iteration algorithm fusionClient       fusionServer       refinement               
1         GES       FusionIntersection FusionIntersection GES        -168605.390014
                                                          None       -168605.390014
                                       FusionUnion        GES        -168605.390014
                                                          None       -168605.390014
                    FusionUnion        FusionIntersection GES        -168605.390014
                                                          None       -168605.390014
                                       FusionUnion        GES        -168605.390014
                                                          None       -168605.390014
          pGES      FusionIntersection FusionIntersection GES        -168489.160193
                                                          None       -168487.103969
                                       FusionUnion        GES        -168489.160193
                                                          None       -168489.160193
                    FusionUnion        FusionIntersection GES        -168489.160193
                                                          None       -168489.160193
                                       FusionUnion        GES        -168489.160193
                                                          None       -168489.160193
5         GES       FusionIntersection FusionIntersection GES        -168436.082941
                                                          None       -168514.262118
                                       FusionUnion        GES        -168328.721539
                                                          None       -168330.966689
                    FusionUnion        FusionIntersection GES        -168471.445652
                                                          None       -168320.067158
                                       FusionUnion        GES        -168328.721539
                                                          None       -168500.997960
          pGES      FusionIntersection FusionIntersection GES        -168383.840722
                                                          None       -168283.991719
                                       FusionUnion        GES        -168394.576569
                                                          None       -168379.685017
                    FusionUnion        FusionIntersection GES        -168375.548885
                                                          None       -168375.542480
                                       FusionUnion        GES        -168386.938561
                                                          None       -168398.307589

### SMHD

In [41]:
df_unstack = df2.drop(["sample", "id", "threads", "instances", "epoch", "bdeu", "time(s)"], axis=1)
df_unstack.groupby(["bbdd", "algorithm", "fusionClient", "fusionServer", "refinement", "iteration"], observed=True).mean().unstack(0)

SMHD  \
bbdd                                                                  alarm   
algorithm fusionClient       fusionServer       refinement iteration          
GES       FusionIntersection FusionIntersection GES        1           9.25   
                                                           5           7.00   
                                                None       1           9.25   
                                                           5           7.00   
                             FusionUnion        GES        1           9.25   
                                                           5           7.25   
                                                None       1           9.25   
                                                           5           7.00   
          FusionUnion        FusionIntersection GES        1           9.25   
                                                           5           7.25   
                                                None       1           9.25   
                                                           5           7.00   
                             FusionUnion        GES        1           9.25   
                                                           5           7.25   
                                                None       1           9.25   
                                                           5          11.00   
pGES      FusionIntersection FusionIntersection GES        1           7.50   
                                                           5           7.00   
                                                None       1           7.25   
                                                           5           7.00   
                             FusionUnion        GES        1           7.50   
                                                           5           7.50   
                                                None       1           7.50   
                                                           5           7.00   
          FusionUnion        FusionIntersection GES        1           7.50   
                                                           5           7.00   
                                                None       1           7.50   
                                                           5           7.00   
                             FusionUnion        GES        1           7.50   
                                                           5           7.50   
                                                None       1           7.50   
                                                           5           7.25   

                                                                              \
bbdd                                                                   andes   
algorithm fusionClient       fusionServer       refinement iteration           
GES       FusionIntersection FusionIntersection GES        1          148.50   
                                                           5          148.00   
                                                None       1          148.50   
                                                           5          145.00   
                             FusionUnion        GES        1          148.50   
                                                           5          148.00   
                                                None       1          148.50   
                                                           5          148.00   
          FusionUnion        FusionIntersection GES        1          148.50   
                                                           5          148.00   
                                                None       1          148.50   
                                                           5          148.00   
                             FusionUnion        GES        1          148.50   
                         

In [42]:
df_unstack = df2.drop(["sample", "id", "threads", "instances", "epoch", "bdeu", "time(s)", "fusionClient", "fusionServer", "refinement"], axis=1)
df_unstack.groupby(["bbdd", "algorithm", "iteration"], observed=True).mean().unstack(0)

SMHD                                                 \
bbdd                   alarm      andes barley    child hailfinder   hepar2   
algorithm iteration                                                           
GES       1          9.25000  148.50000   60.5  8.25000    34.7500  76.2500   
          5          7.59375  145.84375   56.0  1.71875    35.5625  76.3125   
pGES      1          7.46875  148.00000   58.5  3.75000    34.5000  76.2500   
          5          7.15625  145.84375   57.5  2.06250    33.8750  76.1250   

                                                        
bbdd                insurance mildew   water  win95pts  
algorithm iteration                                     
GES       1           22.2500  41.00  90.000  56.00000  
          5           21.1875  41.00  88.500  49.59375  
pGES      1           25.2500  44.00  91.750  49.00000  
          5           25.1250  43.25  88.125  48.75000

In [43]:
df_unstack = df2.drop(["sample", "id", "threads", "instances", "epoch", "bdeu", "time(s)","bbdd"], axis=1)
df_unstack.groupby(["iteration", "algorithm", "fusionClient", "fusionServer", "refinement"], observed=True).mean()

SMHD
iteration algorithm fusionClient       fusionServer       refinement        
1         GES       FusionIntersection FusionIntersection GES         54.675
                                                          None        54.675
                                       FusionUnion        GES         54.675
                                                          None        54.675
                    FusionUnion        FusionIntersection GES         54.675
                                                          None        54.675
                                       FusionUnion        GES         54.675
                                                          None        54.675
          pGES      FusionIntersection FusionIntersection GES         53.850
                                                          None        53.825
                                       FusionUnion        GES         53.850
                                                          None        53.850
                    FusionUnion        FusionIntersection GES         53.850
                                                          None        53.850
                                       FusionUnion        GES         53.850
                                                          None        53.850
5         GES       FusionIntersection FusionIntersection GES         52.950
                                                          None        51.700
                                       FusionUnion        GES         52.100
                                                          None        52.650
                    FusionUnion        FusionIntersection GES         53.200
                                                          None        52.225
                                       FusionUnion        GES         52.100
                                                          None        51.725
          pGES      FusionIntersection FusionIntersection GES         53.775
                                                          None        50.575
                                       FusionUnion        GES         53.200
                                                          None        52.700
                    FusionUnion        FusionIntersection GES         53.525
                                                          None        53.500
                                       FusionUnion        GES         52.925
                                                          None        52.050

### Time(s)

In [44]:
df_unstack = df2.drop(["sample", "id", "threads", "instances", "epoch", "SMHD", "bdeu"], axis=1)
df_unstack.groupby(["bbdd", "algorithm", "fusionClient", "fusionServer", "refinement", "iteration"], observed=True).mean().unstack(0)

time(s)  \
bbdd                                                                    alarm   
algorithm fusionClient       fusionServer       refinement iteration            
GES       FusionIntersection FusionIntersection GES        1          0.16825   
                                                           5          0.03575   
                                                None       1          0.14125   
                                                           5          0.03925   
                             FusionUnion        GES        1          0.14525   
                                                           5          0.04050   
                                                None       1          0.15175   
                                                           5          0.03225   
          FusionUnion        FusionIntersection GES        1          0.19125   
                                                           5          0.04300   
                                                None       1          0.14050   
                                                           5          0.03450   
                             FusionUnion        GES        1          0.13650   
                                                           5          0.04350   
                                                None       1          0.12850   
                                                           5          0.04000   
pGES      FusionIntersection FusionIntersection GES        1          0.15100   
                                                           5          0.07300   
                                                None       1          0.15375   
                                                           5          0.06775   
                             FusionUnion        GES        1          0.15125   
                                                           5          0.07400   
                                                None       1          0.18650   
                                                           5          0.06750   
          FusionUnion        FusionIntersection GES        1          0.14625   
                                                           5          0.07400   
                                                None       1          0.21775   
                                                           5          0.08900   
                             FusionUnion        GES        1          0.16575   
                                                           5          0.07400   
                                                None       1          0.39125   
                                                           5          0.15800   

                                                                                \
bbdd                                                                     andes   
algorithm fusionClient       fusionServer       refinement iteration             
GES       FusionIntersection FusionIntersection GES        1          15.76825   
                                                           5           0.63575   
                                                None       1          15.69675   
                                                           5           1.52550   
                             FusionUnion        GES        1          17.09100   
                                                           5           0.62675   
                                                None       1          15.40700   
                                                           5           0.71450   
          FusionUnion        FusionIntersection GES        1          15.82100   
                                                           5           0.73725   
                                                None       1          15.32800   
                                                           5           0.92200   
    

In [45]:
df_unstack = df2.drop(["sample", "id", "threads", "instances", "epoch", "bdeu", "SMHD", "fusionClient", "fusionServer", "refinement"], axis=1)
df_unstack.groupby(["bbdd", "algorithm", "iteration"], observed=True).mean().unstack(0)

time(s)                                            \
bbdd                    alarm      andes    barley     child hailfinder   
algorithm iteration                                                       
GES       1          0.150406  15.818812  0.911781  0.025719   0.346000   
          5          0.038594   0.808406  0.348406  0.013219   0.090656   
pGES      1          0.195437   3.666656  0.832562  0.062437   0.245031   
          5          0.084656   1.928656  0.419156  0.032219   0.188219   

                                                                       
bbdd                   hepar2 insurance    mildew     water  win95pts  
algorithm iteration                                                    
GES       1          0.531969  0.069781  0.781219  0.060062  0.920875  
          5          0.103063  0.019031  0.554906  0.024594  0.129531  
pGES      1          0.316437  0.093406  0.663219  0.120750  0.727031  
          5          0.234563  0.053344  0.519219  0.070625  0.263688